In [ ]:
import pandas as pd
from pathlib import Path
import json

# ============================================
# 0. 경로 설정
# ============================================
BASE_DIR = Path(".")

# 입력 코호트 (CCI까지 포함된 버전)
COHORT_PATH = BASE_DIR / "cohort" / "cohort_ver46_with_cci.csv"

# MIMIC-IV diagnoses_icd
DIAGNOSES_PATH = (
    BASE_DIR
    / ".."
    / ".."
    / "data"
    / "MIMIC4-hosp-icu"
    / "diagnoses_icd.csv"
)

# HFRS ICD 매핑 파일 (HFRS_code.csv 사용)
# ref/HFRS_code.csv 위치에 두기
HFRS_MAP_PATH = BASE_DIR / "ref" / "HFRS_code.csv"

# SOFA 점수 파일 (stay_id별 첫 24h/최대 SOFA + 서브스코어)
SOFA_SCORES_PATH = (
    BASE_DIR
    / ".."
    / ".."
    / "data"
    / "MIMIC4-derived"
    / "sofa_icu_scores.csv"
)

# 출력 코호트
OUTPUT_PATH = BASE_DIR / "cohort" / "cohort_ver47_with_cci_hfrs_sofa.csv"


# ============================================
# 1. 공통 유틸: ICD 코드 정규화
# ============================================
def normalize_icd(code: str) -> str:
    """
    ICD 코드에서 공백 제거, 대문자 변환, '.' 제거.
    접두(prefix) 매칭용 정규화 함수.
    """
    if pd.isna(code):
        return ""
    s = str(code).strip().upper().replace(".", "")
    return s


# ============================================
# 2. HFRS 계산 관련 함수
# ============================================
def load_hfrs_map(path: Path) -> pd.DataFrame:
    """
    HFRS_code.csv 형식의 HFRS ICD 매핑 파일 로드 및 정규화.
    기대 컬럼:
      - icd_code
      - points

    내부적으로:
      - icd_code_prefix (→ icd_code)
      - weight (→ points)
      - icd_version (모두 10으로 설정)
    """
    print(f"[HFRS] Loading HFRS ICD map: {path.resolve()}")

    if not path.exists():
        raise FileNotFoundError(f"HFRS ICD map not found at: {path.resolve()}")

    hfrs_map = pd.read_csv(path)

    # 컬럼 이름을 기존 로직에 맞게 변환
    if "icd_code" not in hfrs_map.columns or "points" not in hfrs_map.columns:
        raise ValueError("HFRS_code.csv에는 'icd_code', 'points' 컬럼이 필요합니다.")

    hfrs_map = hfrs_map.rename(
        columns={
            "icd_code": "icd_code_prefix",
            "points": "weight",
        }
    )

    # HFRS는 ICD-10 기반으로 간주
    hfrs_map["icd_version"] = 10

    # 정규화된 prefix
    hfrs_map["icd_code_prefix_norm"] = (
        hfrs_map["icd_code_prefix"]
        .astype(str)
        .str.strip()
        .str.upper()
        .str.replace(".", "", regex=False)
    )

    print("[HFRS] Map rows:", len(hfrs_map))
    return hfrs_map


def map_icd_to_hfrs(dx_df: pd.DataFrame, hfrs_map: pd.DataFrame) -> pd.DataFrame:
    """
    diagnoses_icd 서브셋에 대해 HFRS 매핑을 적용해
    hadm_id - icd_code_norm - weight 테이블을 생성.

    dx_df: columns - hadm_id, icd_version, icd_code_norm
    hfrs_map: columns - icd_version, icd_code_prefix_norm, weight
    """
    dx_work = dx_df[["hadm_id", "icd_version", "icd_code_norm"]].copy()
    results = []

    # prefix 길이 (길이가 긴 prefix부터 시도)
    for plen in [5, 4, 3]:
        tmp = dx_work.copy()
        tmp["prefix"] = tmp["icd_code_norm"].str[:plen]

        map_sub = hfrs_map.copy()
        map_sub["prefix"] = map_sub["icd_code_prefix_norm"].str[:plen]

        merged = tmp.merge(
            map_sub[["icd_version", "prefix", "weight"]],
            on=["icd_version", "prefix"],
            how="inner",
        )

        # 동일 hadm_id / 코드에 대한 중복 제거
        merged = merged[["hadm_id", "icd_code_norm", "weight"]].drop_duplicates()
        results.append(merged)

    if not results:
        return pd.DataFrame(columns=["hadm_id", "icd_code_norm", "weight"])

    mapped = pd.concat(results, ignore_index=True).drop_duplicates()
    return mapped


def compute_hfrs_for_cohort(cohort: pd.DataFrame) -> pd.DataFrame:
    """
    코호트(hadm_id 기준)에 대해 HFRS 점수와 카테고리를 계산해
    hadm_id - hfrs_score - hfrs_category 테이블을 반환.
    """
    print("[HFRS] Loading diagnoses_icd...")
    dx = pd.read_csv(DIAGNOSES_PATH)
    print("[HFRS] diagnoses_icd rows:", len(dx))

    hadm_ids = cohort["hadm_id"].unique()
    dx_sub = dx[dx["hadm_id"].isin(hadm_ids)].copy()
    print("[HFRS] diagnoses_icd rows (in cohort):", len(dx_sub))

    # ICD 정규화
    dx_sub["icd_code_norm"] = dx_sub["icd_code"].apply(normalize_icd)

    # HFRS 매핑 로드
    hfrs_map = load_hfrs_map(HFRS_MAP_PATH)

    # ICD → HFRS weight 매핑
    dx_hfrs = map_icd_to_hfrs(dx_sub, hfrs_map)
    print("[HFRS] Mapped HFRS rows:", len(dx_hfrs))

    if dx_hfrs.empty:
        # 매핑이 하나도 안 된 경우
        hfrs_table = pd.DataFrame({"hadm_id": hadm_ids})
        hfrs_table["hfrs_score"] = 0.0
    else:
        # hadm_id별 HFRS 점수 = 해당 admission ICD들의 weight 합
        hfrs_table = (
            dx_hfrs.groupby("hadm_id", as_index=False)["weight"]
            .sum()
            .rename(columns={"weight": "hfrs_score"})
        )

    # 카테고리화: <5 / 5–15 / >15
    def categorize_hfrs(score: float) -> str:
        """
        Hospital Frailty Risk Score 카테고리:
          - score < 5       → "lt5"
          - 5 <= score <=15 → "5_15"
          - score > 15      → "gt15"
        """
        if pd.isna(score):
            return "lt5"
        s = float(score)
        if s < 5:
            return "lt5"
        elif 5 <= s <= 15:
            return "5_15"
        else:
            return "gt15"

    hfrs_table["hfrs_category"] = hfrs_table["hfrs_score"].apply(categorize_hfrs)

    print("[HFRS] Example rows:")
    print(hfrs_table.head())

    return hfrs_table


# ============================================
# 3. SOFA 점수 병합 (첫 24시간, 최대, 서브스코어)
# ============================================
def load_sofa_scores(path: Path) -> pd.DataFrame:
    """
    SOFA 점수 파일 로드 및 컬럼 정리.

    기대 컬럼:
      - stay_id
      - sofa_icu_adm_score (ICU 입실 후 첫 24h SOFA 총점)
      - sofa_icu_max_score (ICU 체류 중 최대 SOFA 총점)
      - sofa_resp, sofa_coag, sofa_liver, sofa_cardio, sofa_cns, sofa_renal
    """
    print(f"[SOFA] Loading SOFA scores: {path}")
    df = pd.read_csv(path)

    # 컬럼 이름 보정 예시
    if "sofa_icu_adm_score" not in df.columns and "sofa_24hours" in df.columns:
        df = df.rename(columns={"sofa_24hours": "sofa_icu_adm_score"})
    if "sofa_icu_max_score" not in df.columns and "sofa_max" in df.columns:
        df = df.rename(columns={"sofa_max": "sofa_icu_max_score"})

    required_cols = [
        "stay_id",
        "sofa_icu_adm_score",
        "sofa_icu_max_score",
        "sofa_resp",
        "sofa_coag",
        "sofa_liver",
        "sofa_cardio",
        "sofa_cns",
        "sofa_renal",
    ]
    missing = [c for c in required_cols if c not in df.columns]
    if missing:
        raise ValueError(f"SOFA 파일에 필요한 컬럼이 없습니다: {missing}")

    # stay_id 중복이 있다면 최대값 기준으로 집계
    agg_dict = {
        "sofa_icu_adm_score": "max",
        "sofa_icu_max_score": "max",
        "sofa_resp": "max",
        "sofa_coag": "max",
        "sofa_liver": "max",
        "sofa_cardio": "max",
        "sofa_cns": "max",
        "sofa_renal": "max",
    }
    df_agg = df.groupby("stay_id", as_index=False).agg(agg_dict)

    print("[SOFA] Aggregated rows:", len(df_agg))
    print(df_agg.head())
    return df_agg


def add_sofa_subscores(df: pd.DataFrame) -> pd.DataFrame:
    """
    서브스코어 6개(호흡/혈액/간/심혈관/신경/신장)를
    하나의 JSON 문자열 컬럼 'sofa_subscores'로 묶는다.
    """
    organ_cols = [
        "sofa_resp",
        "sofa_coag",
        "sofa_liver",
        "sofa_cardio",
        "sofa_cns",
        "sofa_renal",
    ]

    def make_json(row):
        data = {k: row[k] for k in organ_cols}
        return json.dumps(data)

    df["sofa_subscores"] = df.apply(make_json, axis=1)
    return df


# ============================================
# 4. 메인 파이프라인
# ============================================
def main():
    # 4-1. 코호트 로드
    print("[MAIN] Loading cohort:", COHORT_PATH)
    cohort = pd.read_csv(COHORT_PATH)
    print("[MAIN] Cohort rows:", len(cohort))

    if "hadm_id" not in cohort.columns:
        raise ValueError("코호트에 'hadm_id' 컬럼이 없습니다.")

    # 4-2. ICU stay id 정리: icu_stay_id → stay_id
    if "stay_id" in cohort.columns:
        print("[MAIN] Cohort already has 'stay_id' 컬럼을 사용합니다.")
    elif "icu_stay_id" in cohort.columns:
        print("[MAIN] 'icu_stay_id' 컬럼을 'stay_id'로 사용합니다.")
        cohort["stay_id"] = cohort["icu_stay_id"]
    else:
        raise ValueError("코호트에 'stay_id' 또는 'icu_stay_id' 컬럼이 없습니다. SOFA 병합 불가.")

    # 4-3. HFRS 계산 (hadm_id 기준)
    hfrs_table = compute_hfrs_for_cohort(cohort)

    # 4-4. SOFA 점수 로드 및 서브스코어 JSON 생성 (파일이 없으면 스킵)
    sofa_scores = None
    try:
        sofa_scores = load_sofa_scores(SOFA_SCORES_PATH)
        sofa_scores = add_sofa_subscores(sofa_scores)
        sofa_available = True
    except FileNotFoundError:
        print(f"[SOFA] SOFA 파일을 찾을 수 없습니다: {SOFA_SCORES_PATH.resolve()}")
        print("[SOFA] SOFA 병합을 건너뛰고, 관련 컬럼은 NaN으로 채웁니다.")
        sofa_available = False

    # 4-5. 코호트에 HFRS merge (hadm_id 기준)
    cohort = cohort.merge(
        hfrs_table[["hadm_id", "hfrs_score", "hfrs_category"]],
        on="hadm_id",
        how="left",
    )

    # hfrs_score 결측 → 0.0, hfrs_category 결측 → "lt5"
    cohort["hfrs_score"] = cohort["hfrs_score"].fillna(0.0)
    cohort["hfrs_category"] = cohort["hfrs_category"].fillna("lt5")

    # 4-6. 코호트에 SOFA merge (stay_id 기준) – 파일 있을 때만
    if sofa_available:
        cohort = cohort.merge(
            sofa_scores[
                [
                    "stay_id",
                    "sofa_icu_adm_score",
                    "sofa_icu_max_score",
                    "sofa_subscores",
                ]
            ],
            on="stay_id",
            how="left",
        )
    else:
        # 컬럼만 만들어 두기
        cohort["sofa_icu_adm_score"] = pd.NA
        cohort["sofa_icu_max_score"] = pd.NA
        cohort["sofa_subscores"] = pd.NA

    print("[MAIN] Final cohort shape:", cohort.shape)
    print(
        cohort[
            [
                "hadm_id",
                "hfrs_score",
                "hfrs_category",
                "stay_id",
                "sofa_icu_adm_score",
                "sofa_icu_max_score",
                "sofa_subscores",
            ]
        ].head()
    )

    # 4-7. 저장
    OUTPUT_PATH.parent.mkdir(parents=True, exist_ok=True)
    cohort.to_csv(OUTPUT_PATH, index=False, encoding="utf-8")
    print("[MAIN] Saved:", OUTPUT_PATH)


if __name__ == "__main__":
    main()


[MAIN] Loading cohort: cohort\cohort_ver46_with_cci.csv
[MAIN] Cohort rows: 1930
[MAIN] 'icu_stay_id' 컬럼을 'stay_id'로 사용합니다.
[HFRS] Loading diagnoses_icd...
[HFRS] diagnoses_icd rows: 6364488
[HFRS] diagnoses_icd rows (in cohort): 27892
[HFRS] Loading HFRS ICD map: D:\University\3-2\1Data_Analytics\team\DA_git\DA_STEMI\ref\HFRS_code.csv
[HFRS] Map rows: 109
[HFRS] Mapped HFRS rows: 2745
[HFRS] Example rows:
    hadm_id  hfrs_score hfrs_category
0  20082514         1.2           lt5
1  20090653         4.1           lt5
2  20090705         6.4          5_15
3  20102387         2.2           lt5
4  20121113         2.5           lt5


NameError: name 'f' is not defined